In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sweetviz as sv
import dtale

import config

#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 50)

In [10]:
df_patient=pd.read_pickle(config.INTERIM_PICKLE_EDA_DIR+'/data_clinical_patient_1.0.pkl')
df_sample=pd.read_pickle(config.INTERIM_PICKLE_EDA_DIR+'/data_clinical_sample_1.1.pkl')

In [14]:
df_metabric_table=pd.concat([df_patient, df_sample], axis=1)
df_metabric_table.shape

(2509, 37)

In [15]:
df_metabric_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2509 entries, 0 to 2508
Data columns (total 37 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   PATIENT_ID                           2509 non-null   object 
 1   LYMPH_NODES_EXAMINED_POSITIVE        2243 non-null   float64
 2   NPI                                  2287 non-null   float64
 3   COHORT                               2509 non-null   object 
 4   SEX                                  2509 non-null   object 
 5   INTCLUST                             2509 non-null   object 
 6   AGE_AT_DIAGNOSIS                     2498 non-null   float64
 7   OS_MONTHS                            1981 non-null   float64
 8   OS_STATUS                            1981 non-null   object 
 9   CLAUDIN_SUBTYPE                      2509 non-null   object 
 10  THREEGENE                            2509 non-null   object 
 11  VITAL_STATUS                  

In [26]:
target_columns=[
    # patient
    'OS_MONTHS',
    'OS_STATUS',
    'RFS_STATUS',
    'RFS_MONTHS',
    'VITAL_STATUS',
]

int_columns=[
    # patient
    'LYMPH_NODES_EXAMINED_POSITIVE',
]

float_columns=[
    # patient
    'NPI',
    'AGE_AT_DIAGNOSIS',
    # sample
    'TUMOR_SIZE',
    'TMB_NONSYNONYMOUS',
]

ｓｔｒ_columns=[
]

# 質的変数（順序尺度）
qualitative_ordinal_columns=[
    # patient
    'CELLULARITY',  
    'ER_IHC',
    'HER2_SNP6',
    'INFERRED_MENOPAUSAL_STATE',
    # sample
    'ER_STATUS',
    'HER2_STATUS',
    'GRADE',
    'PR_STATUS',
    'TUMOR_STAGE',
]
# 質的変数（名義尺度）
qualitative_name_columns=[
    # patient
    'COHORT',
    'INTCLUST',
    'CLAUDIN_SUBTYPE',
    'THREEGENE',
    'HISTOLOGICAL_SUBTYPE',
    'BREAST_SURGERY',
    'LATERALITY',
    # sample
    'CANCER_TYPE',
    'CANCER_TYPE_DETAILED',
    'ONCOTREE_CODE',

]

bool_columns=[
    # patient
    'CHEMOTHERAPY',
    'HORMONE_THERAPY',
    'RADIO_THERAPY',
]

meanless_columns=[
    # patient
    'PATIENT_ID',
    'SEX',
    # sample
    'PATIENT_ID',
    'SAMPLE_ID',
    'SAMPLE_TYPE',
]

print(df_metabric_table.shape[1],
    len(target_columns)+\
    len(int_columns)+\
    len(float_columns)+\
    len(ｓｔｒ_columns)+\
    len(qualitative_ordinal_columns)+\
    len(qualitative_name_columns)+\
    len(bool_columns)+\
    len(meanle ss_columns))
assert  df_metabric_table.shape[1]==\
    len(target_columns)+\
    len(int_columns)+\
    len(float_columns)+\
    len(ｓｔｒ_columns)+\
    len(qualitative_ordinal_columns)+\
    len(qualitative_name_columns)+\
    len(bool_columns)+\
    len(meanless_columns), 'lack or too much columns'

37 37
